In [1]:
import os
import random
import wikipedia
import concurrent.futures
from tqdm import tqdm

# output_wikiディレクトリを作成
os.makedirs("output_wiki", exist_ok=True)

# 全ての英語Wikipediaページのタイトルを取得する
all_titles = list(wikipedia.random(pages=5000))  # 5000ページ分のタイトルを取得

# 100個のタイトルをランダムに選択する
selected_titles = random.sample(all_titles, 100)

def download_page(args):
    i, title = args
    try:
        page = wikipedia.page(title)
    except wikipedia.exceptions.PageError:
        # 記事が見つからない場合はスキップ
        return
    except wikipedia.exceptions.DisambiguationError as e:
        # 曖昧な記事はスキップする
        print(f"Skipped disambiguation page: {e.options}")
    except Exception as e:
        print(f"Error: {e}")
    else:
        filename = os.path.join("output_wiki", f"{i:03d}.txt")
        with open(filename, "w", encoding="utf-8") as f:
            f.write(page.content)
        print(f"Saved article: {page.title}")

# 選択したタイトルの記事を取得し、保存する
with concurrent.futures.ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(download_page, enumerate(selected_titles, start=1)), total=len(selected_titles)))


KeyboardInterrupt: 

In [5]:
import re
import nltk
from nltk.tokenize import sent_tokenize

# NLTKのデータをダウンロード
nltk.download('punkt')

# output_wikiディレクトリのパス
wiki_dir = "output_wiki"

# output_textディレクトリを作成
output_dir = "output_text"
os.makedirs(output_dir, exist_ok=True)

# output_wikiディレクトリ内のファイルを処理
for filename in os.listdir(wiki_dir):
    # ファイルパスを構築
    file_path = os.path.join(wiki_dir, filename)

    # ファイルを読み込む
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()

    # 記事を1文ずつに分割
    sentences = sent_tokenize(content)

    # output_textディレクトリ内にサブディレクトリを作成
    sub_dir_name = os.path.splitext(filename)[0]
    sub_dir_path = os.path.join(output_dir, sub_dir_name)
    os.makedirs(sub_dir_path, exist_ok=True)

    # 分割した文をファイルに保存
    for i, sentence in enumerate(sentences, start=1):
        sentence_filename = os.path.join(sub_dir_path, f"{i:03d}.txt")
        with open(sentence_filename, "w", encoding="utf-8") as f:
            f.write(sentence)

print("文の分割が完了しました。")

[nltk_data] Downloading package punkt to /home/souta-pqr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


文の分割が完了しました。


In [4]:
import os
import textwrap
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from tqdm import tqdm  # プログレスバーを表示するためのライブラリ

# 翻訳モデルの設定
model_name = 'facebook/mbart-large-50-many-to-many-mmt'
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

# output_textディレクトリのパス
input_dir = "output_text"

# output_japanese_textディレクトリを作成
output_dir = "output_japanese_text"
os.makedirs(output_dir, exist_ok=True)

# サブディレクトリの総数を取得
total_subdirs = len(os.listdir(input_dir))

# output_textディレクトリ内のサブディレクトリを処理
for i, sub_dir_name in enumerate(os.listdir(input_dir), start=1):
    sub_dir_path = os.path.join(input_dir, sub_dir_name)

    # output_japanese_textディレクトリ内にサブディレクトリを作成
    output_sub_dir_path = os.path.join(output_dir, sub_dir_name)
    os.makedirs(output_sub_dir_path, exist_ok=True)

    # サブディレクトリ内のファイルを処理
    files = os.listdir(sub_dir_path)
    for filename in tqdm(files, desc=f"Processing subdirectory {i}/{total_subdirs}", unit="file"):
        file_path = os.path.join(sub_dir_path, filename)

        # ファイルを読み込む
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        # 文章を小さな部分に分割
        sentences = textwrap.wrap(content, width=100)

        translated_sentences = []
        for sent in sentences:
            # 文章をトークナイザーでトークナイズし、モデルが理解できる形式に変換
            inputs = tokenizer(sent, return_tensors="pt")

            # 翻訳の実行
            generated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["ja_XX"])
            translated = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

            translated_sentences.append(translated[0])

        # 翻訳された内容をファイルに保存
        output_file_path = os.path.join(output_sub_dir_path, filename)
        with open(output_file_path, "w", encoding="utf-8") as f:
            for translation in translated_sentences:
                f.write(translation + "\n")

print("翻訳が完了しました。")


Processing subdirectory 1/100:  67%|██████▋   | 2/3 [00:08<00:04,  4.25s/file]


KeyboardInterrupt: 

In [2]:
import os
from tqdm import tqdm
from espnet2.bin.tts_inference import Text2Speech
import soundfile as sf

# モデル名
model_tag = "kan-bayashi/jsut_full_band_vits_prosody"
vocoder_tag = "parallel_wavegan/jsut_parallel_wavegan.v1"

# 音声合成器の生成
text2speech = Text2Speech.from_pretrained(
    model_tag=model_tag,
    vocoder_tag=vocoder_tag,
)

# output_japanese_textディレクトリのパス
input_dir = "output_japanese_text"

# output_audioディレクトリを作成
output_dir = "output_audio"
os.makedirs(output_dir, exist_ok=True)

# output_japanese_textディレクトリ内のサブディレクトリを処理
for sub_dir_name in os.listdir(input_dir):
    sub_dir_path = os.path.join(input_dir, sub_dir_name)
    
    # output_audioディレクトリ内にサブディレクトリを作成
    output_sub_dir_path = os.path.join(output_dir, sub_dir_name)
    os.makedirs(output_sub_dir_path, exist_ok=True)
    
    # サブディレクトリ内のファイルを処理
    files = os.listdir(sub_dir_path)
    for i, filename in enumerate(tqdm(sorted(files), desc=f"Processing {sub_dir_name}", unit="file"), start=1):
        file_path = os.path.join(sub_dir_path, filename)
        
        # ファイルを読み込む
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
        
        # 音声合成
        result = text2speech(content)
        
        # 音声ファイルの保存
        output_file_path = os.path.join(output_sub_dir_path, f"{i:03}.wav")
        sf.write(output_file_path, result["wav"], text2speech.fs, "PCM_16")

print("音声ファイルの作成が完了しました。")

/home/souta-pqr/anaconda3/envs/w2t/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
https://zenodo.org/record/5521340/files/tts_train_full_band_vits_raw_phn_jaconv_pyopenjtalk_prosody_train.total_count.ave.zip?download=1: 100%|██████████| 357M/357M [19:23<00:00, 322kB/s]    
/home/souta-pqr/anaconda3/envs/w2t/lib/python3.9/site-packages/espnet_model_zoo/downloader.py:369: UserWarning: Not validating checksum
  warnings.warn("Not validating checksum")
/home/souta-pqr/anaconda3/envs/w2t/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/home/souta-pqr/anaconda3/envs/w2t/l

Downloading: "https://github.com/r9y9/open_jtalk/releases/download/v1.11.1/open_jtalk_dic_utf_8-1.11.tar.gz"


dic.tar.gz: 100%|██████████| 22.6M/22.6M [00:03<00:00, 7.03MB/s]


Extracting tar file /home/souta-pqr/anaconda3/envs/w2t/lib/python3.9/site-packages/pyopenjtalk/dic.tar.gz


Processing 080:  20%|██        | 1/5 [00:25<01:40, 25.11s/file]


KeyboardInterrupt: 